In [1]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

import sys
import numpy as np

In [2]:
a = np.random.randn(4,4)

a = a.astype(np.float32)

In [3]:
a_gpu = cuda.mem_alloc(a.nbytes)

In [4]:
cuda.memcpy_htod(a_gpu, a)

In [7]:
mod = SourceModule("""
  __global__ void doublify(float *a)
  {
    int idx = threadIdx.x + threadIdx.y*4;
    a[idx] *= 2;
  }
  """)

In [8]:
func = mod.get_function("doublify")
func(a_gpu, block=(4,4,1))

In [10]:
a_doubled = np.empty_like(a)
cuda.memcpy_dtoh(a_doubled, a_gpu)
print(a_doubled)
print(a)

[[-1.1461985   0.1649619  -1.9914348  -2.553466  ]
 [ 3.2260573  -2.309934    1.3971603   5.9942307 ]
 [-0.5852058   2.6254065   0.34095192 -1.2509546 ]
 [-1.6057737  -0.9900442  -1.0163801  -1.5364295 ]]
[[-0.57309926  0.08248095 -0.9957174  -1.276733  ]
 [ 1.6130286  -1.154967    0.69858015  2.9971154 ]
 [-0.2926029   1.3127033   0.17047596 -0.6254773 ]
 [-0.80288684 -0.4950221  -0.50819004 -0.76821476]]
